In [ ]:
%run /Utils/Functions/core

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from pyspark.sql.functions import col
from uuid import uuid4
import time
from datetime import datetime, timedelta, timezone
import concurrent.futures
from time import sleep, mktime
import json

In [ ]:
## Cria as variáveis de LOG
format_log = '%Y-%m-%d %H:%M:%S'
dtInicio = datetime.today() - timedelta(hours=3)
dtInicio_format = dtInicio.strftime(format_log)
tipo_log = 'API'
camada = 'landing'
emissor = '<org>'
atividade = '<activity_desc>'
origem = 'RestService'
destino = 'AzureBlobFS'
execUrl = ' '

try:
    infos = json.loads(dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson()) # captura as informações do job que executa o notebook
    orgId = infos['tags']['orgId']
    runId = infos['tags']['multitaskParentRunId']
    jobId = infos['tags']['jobId']
    if orgId == '2960871991268730': # Monta a URL caso seja o ID do ambiente de DEV
        execUrl = f'https://adb-{orgId}.10.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
    else: # Monta a URL caso seja o ID do ambiente de PROD
        execUrl = f'https://adb-{orgId}.15.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
except:
    print('Campo de URL não pode ser identificado!')

In [ ]:
%sql
use catalog prod;

In [ ]:
dbutils.widgets.text("dt_ingestao", "")

dt_ingestao = getArgument("dt_ingestao").upper().strip()

location_landing = spark.sql("show external locations").select("url").where("name = 'landing-area'").collect()[0][0]

In [ ]:
# Formata o dt_ingestao
format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
dt_ingestao = datetime.now() if dt_ingestao == "" else datetime.strptime(dt_ingestao, format_timestamp)

# Pega o horário atual e tira 3 horas para converter para BRT (UTC -3)
# Tira 5 minutos por conta de um limite da API
dt_fim = dt_ingestao - timedelta(hours=3) - timedelta(minutes=5)
dt_inicio = dt_fim - timedelta(days=1)

# Força as horas, minutos e segundos em 0
dt_inicio = datetime(dt_inicio.year, dt_inicio.month, dt_inicio.day, 0, 0, 0)

# Separa as datas no formato UNIX
dt_fim_unix = int(mktime(dt_fim.timetuple()))
dt_inicio_unix = int(mktime(dt_inicio.timetuple()))

# dt_inicio_unix = 1546300800

# Transforma as datas em STRING utilizando um formato de timestamp
dt_fim = dt_fim.strftime(format_timestamp)
dt_inicio = dt_inicio.strftime(format_timestamp)

print(f"dt_inicio: {dt_inicio} | unix: {dt_inicio_unix}")
print(f"dt_fim   : {dt_fim} | unix: {dt_fim_unix}")

##CONFIGURAÇÕES DE PROCESSO DE CARGA

In [ ]:
user = dbutils.secrets.get("scope-vault-data", "jira-user")
password  = dbutils.secrets.get("scope-vault-data", "jira-api-token")
domain = "<domain>"
subdomain = "atlassian"
baseUrl = f"https://{domain}.{subdomain}.net"
auth = HTTPBasicAuth(f"{user}", f"{password}")

threads = 10    

In [ ]:
df_param = fn_ConsultaJdbc("""
    SELECT pca.*
    FROM ctl.ADF_Parametro_Carga_API pca
    WHERE pca.fl_ativo = 1
    AND nm_Sistema = 'jira'
    AND (ds_Custom_Field != 'id' or ds_Custom_Field is null)
    AND pca.id_Parametro_Carga_API not in (579)
""")

display(df_param)
data_param = df_param.collect()

In [ ]:
df_param_id = fn_ConsultaJdbc("""
    SELECT pca.*
    FROM ctl.ADF_Parametro_Carga_API pca
    WHERE pca.fl_ativo = 1
    AND nm_Sistema = 'jira' and ds_Custom_Field = 'id'
""")

display(df_param_id)
data_param_id = df_param_id.collect()

## SCRIPT DA API

In [ ]:
def get_tables_from_ids(param, baseUrl, dt_inicio_unix, dtIngestao, location_landing, ids, id_tabela):
    end_point_page = None
    try:
        # Declarando as variáveis
        inicio = time.time()

        if param.vl_Schedule_Carga != 0:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
        else:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()
        
        data_format = []
        for id in ids:
            start_at = None
            total = None
            is_last = None
            max_results = None
            pagina = 1
            print(f"Tabela {param.ds_Nome_Arquivo_Landing} ---> ID: {id}! Começando carga...")

            # Pega o endpoint que utilizará no request
            # Verificação para casos especiais que salvam a última URL que processou
            end_point_page = baseUrl+param.ds_Url.replace('@id', str(id))
            req = requests.get(end_point_page, auth=auth)
            data = req.json()
            data_list = []

            # Verifica se a primeira requisição teve erro
            if req.status_code != 200:
                print("\n"+f"Tabela {param.ds_Nome_Arquivo_Landing} com erro na primeira requisição do ID {id}!")
            # Caso a response da requisição seja 200 (sucesso) 
            else:
                # Armazena o conteúdo da API na variável data em formato JSON
                data = req.json()

                if type(data) != dict:
                    data = {}
                    data[param.nm_Item_Origem] = req.json()

                end_point_page = None
                data_keys = list(data.keys())

                # Algumas tabelas salvam a partir de colunas com o nome diferente da tabela
                if param.nm_Item_Origem == 'issue_label':
                    data_list.append(data)
                elif param.nm_Definition is not None:
                    campo_data = param.nm_Definition.split(',')
                    if len(campo_data) == 2:
                        data_list.extend(data[campo_data[0]][campo_data[1]])
                    else:
                        data_list.extend(data[campo_data[0]])
                else:
                    data_list.extend(data[param.nm_Item_Origem])

                if 'startAt' and 'maxResults' in data_keys:
                    start_at = data['startAt']
                    max_results = data['maxResults']
                    start_at += max_results
                    end_point_page = baseUrl+param.ds_Url.replace('@id', str(id))+f'&startAt={start_at}'
                if 'total' in data_keys:
                    total = data['total']
                elif 'isLast' in data_keys:
                    is_last = data['isLast']

                print(f'Tabela {param.nm_Item_Origem} ---> ID: {id} teve {pagina} páginas adicionadas com êxito!')
                    
            while (total != None and (total - start_at) >= 0 or is_last != None and is_last == False):
                req = requests.get(end_point_page, auth=auth)
                data = req.json()
                pagina += 1

                # Verifica se a primeira requisição teve erro
                if req.status_code != 200:
                    print("\n"+f"Tabela {param.ds_Nome_Arquivo_Landing} com erro na página {pagina}!")
                    pagina -= 1
                # Caso a response da requisição seja 200 (sucesso) 
                else:
                    # Armazena o conteúdo da API na variável data em formato JSON
                    data = req.json()

                    if type(data) != dict:
                        data = {}
                        data[param.nm_Item_Origem] = req.json()

                    end_point_page = None

                    data_keys = list(data.keys())

                    # Algumas tabelas salvam a partir de colunas com o nome diferente da tabela
                    if param.nm_Item_Origem == 'issue_label':
                        data_list.append(data)
                    elif param.nm_Definition is not None:
                        campo_data = param.nm_Definition.split(',')
                        if len(campo_data) == 2:
                            data_list.extend(data[campo_data[0]][campo_data[1]])
                        else:
                            data_list.extend(data[campo_data[0]])
                    else:
                        data_list.extend(data[param.nm_Item_Origem])

                    if 'startAt' and 'maxResults' in data_keys:
                        start_at = data['startAt']
                        max_results = data['maxResults']
                        start_at += max_results
                        end_point_page = baseUrl+param.ds_Url.replace('@id', str(id))+f'&startAt={start_at}'
                    if 'total' in data_keys:
                        is_last = None
                        total = data['total']
                    elif 'isLast' in data_keys:
                        total = None
                        is_last = data['isLast']
                    
                    print(f'Tabela {param.nm_Item_Origem} ---> ID: {id} teve {pagina} páginas adicionadas com êxito!')

            for data in data_list:
                data.update({id_tabela: str(id)})
                if param.nm_Item_Origem == 'issue_label':
                    label = data['fields']['labels']
                    data.pop('fields')
                    data.update({'label': label})
                data_format.append(data)

        if len(data_format) == 0: # caso a quantidade de registros seja 0 (vazio), encerra a execução para evitar erros no LOG
            return

        fim = time.time()
        
        # Subtrai a variável "início" pela "fim" para obter o tempo total de execução da tabela e armazena na váriavel "tempo_exec"
        tempo_exec = fim - inicio

        print("\n"+f"A tabela {param.nm_Item_Origem} teve {pagina} páginas carregadas com êxito!")
        print(f"Tamanho total de registros {len(data_format)}")
        print(f"O tempo de execução da tabela {param.nm_Item_Origem} foi {tempo_exec}")

        # PROCESSO DE CARGA DOS DADOS NO BLOB
        fn_SaveJson(data_format, dir_landing, str(param.ds_Nome_Arquivo_Landing).lower())

        fn_AtualizaUltimoIncremento_API(param.id_Parametro_Carga_API, dtIngestao)

        ## LOG SUCESSO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "execUrl": execUrl}

        fn_LogSucceeded(parametros, dt_ingestao.strftime(format_log))

    except Exception as error:
        ## LOG ERRO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '

        if hasattr(error, 'code'): # captura o código do erro, caso possua o atributo 'code'
            error_code = error.code
        else:
            error_code = 'NULL'
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "cd_erro": error_code, "erro": str(error), "execUrl": execUrl}

        fn_LogFailed(parametros, dt_ingestao.strftime(format_log))

In [ ]:
def get_tables(param, baseUrl, dt_inicio_unix, dtIngestao, location_landing):
    end_point_page = None
    try:
        format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
        dtIngestao = str(dtIngestao)
        if param.vl_Ultimo_Incremento is not None:
            vlUltimoIncremento = datetime.strptime(param.vl_Ultimo_Incremento, format_timestamp)
            vlUltimoIncremento = int(mktime(vlUltimoIncremento.timetuple()))
        else:
            vlUltimoIncremento = dt_inicio_unix

        vlUltimoIncremento = str(vlUltimoIncremento)

        # Declarando as variáveis
        start_at = None
        total = None
        is_last = None
        max_results = None
        pagina = 1
        inicio = time.time()

        if param.vl_Schedule_Carga != 0:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
        else:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()

        print(f"Tabela {param.ds_Nome_Arquivo_Landing}! Começando carga...")

        data_list = []

        # Pega o endpoint que utilizará no request
        # Verificação para casos especiais que salvam a última URL que processou
        if param.ds_Custom_Field is None:
            end_point_page = f"{baseUrl}{param.ds_Url}"
        else:
            end_point_page = baseUrl+param.ds_Custom_Field
            # end_point_page = f"{baseUrl}{param.ds_Url}"
        
        req = requests.get(end_point_page, auth=auth)

        # Verifica se a primeira requisição teve erro
        if req.status_code != 200:
            print("\n"+f"Tabela {param.ds_Nome_Arquivo_Landing} com erro na primeira requisição!")
        # Caso a response da requisição seja 200 (sucesso) 
        else:
            # Armazena o conteúdo da API na variável data em formato JSON
            data = req.json()

            if type(data) != dict:
                data = {}
                data[param.nm_Item_Origem] = req.json()
                
            data_keys = list(data.keys())

            end_point_page = None

            # Algumas tabelas salvam a partir de colunas com o nome diferente da tabela
            if param.nm_Definition is not None:
                data_list.extend(data[param.nm_Definition])
            else:
                data_list.extend(data[param.nm_Item_Origem])

            if len(data_list) == 0: # caso a quantidade de registros seja 0 (vazio), encerra a execução para evitar erros no LOG
                return

            if 'startAt' and 'maxResults' in data_keys:
                start_at = data['startAt']
                max_results = data['maxResults']
                start_at += max_results
                if param.ds_Custom_Field is None:
                    end_point_page = baseUrl+param.ds_Url+f'&startAt={start_at}'
                else:
                    end_point_page = baseUrl+param.ds_Custom_Field+f'&startAt={start_at}'
                    # end_point_page = baseUrl+param.ds_Url+f'&startAt={start_at}'
            if 'total' in data_keys:
                total = data['total']
            elif 'isLast' in data_keys:
                is_last = data['isLast']

        while (total != None and (total - start_at) >= 0 or is_last != None and is_last == False):
            req = requests.get(end_point_page, auth=auth)
            data = req.json()
            pagina += 1

            # Verifica se a primeira requisição teve erro
            if req.status_code != 200:
                print("\n"+f"Tabela {param.ds_Nome_Arquivo_Landing} com erro na página {pagina}!")
                pagina -= 1
            # Caso a response da requisição seja 200 (sucesso) 
            else:
                # Armazena o conteúdo da API na variável data em formato JSON
                data = req.json()

                if type(data) != dict:
                    data = {}
                    data[param.nm_Item_Origem] = req.json()

                end_point_page = None

                data_keys = list(data.keys())

                # Algumas tabelas salvam a partir de colunas com o nome diferente da tabela
                if param.nm_Definition is not None:
                    data_list.extend(data[param.nm_Definition])
                else:
                    data_list.extend(data[param.nm_Item_Origem])

                if 'startAt' and 'maxResults' in data_keys:
                    start_at = data['startAt']
                    max_results = data['maxResults']
                    start_at += max_results
                    if param.ds_Custom_Field is None:
                        end_point_page = baseUrl+param.ds_Url+f'&startAt={start_at}'
                    else:
                        end_point_page = baseUrl+param.ds_Custom_Field+f'&startAt={start_at}'
                        # end_point_page = baseUrl+param.ds_Url+f'&startAt={start_at}'
                if 'total' in data_keys:
                    total = data['total']
                elif 'isLast' in data_keys:
                    is_last = data['isLast']

        data_format = data_list

        fim = time.time()
        
        # Subtrai a variável "início" pela "fim" para obter o tempo total de execução da tabela e armazena na váriavel "tempo_exec"
        tempo_exec = fim - inicio

        print("\n"+f"A tabela {param.nm_Item_Origem} teve {pagina} páginas carregadas com êxito!")
        print(f"Tamanho total de registros {len(data_list)}")
        print(f"O tempo de execução da tabela {param.nm_Item_Origem} foi {tempo_exec}")

        # PROCESSO DE CARGA DOS DADOS NO BLOB
        fn_SaveJson(data_format, dir_landing, str(param.ds_Nome_Arquivo_Landing).lower())

        fn_AtualizaUltimoIncremento_API(param.id_Parametro_Carga_API, dtIngestao)

        ## LOG SUCESSO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "execUrl": execUrl}

        fn_LogSucceeded(parametros, dt_ingestao.strftime(format_log))

        # Tabela Board_Issue
        if param.nm_Item_Origem in ['board', 'issue', 'project']:
            list_ids = []
            subtables_board = ['board_issue', 'board_project', 'sprint']
            subtables_issue = ['issue_changelog','issue_comment','issue_link','issue_subtask','issue_watcher','issue_worklog','issue_label','issue_component','issue_fix_version','issue_version','issue_change_details']
            subtables_project = ['version','component']

            if param.nm_Item_Origem == 'board':
                data_param_id = df_param_id.filter(col('nm_Item_Origem').isin(subtables_board)).collect()
                id_tabela = 'board_id'
            elif param.nm_Item_Origem == 'issue':
                data_param_id = df_param_id.filter(col('nm_Item_Origem').isin(subtables_issue)).collect()
                id_tabela = 'issue_id'
            elif param.nm_Item_Origem == 'project':
                data_param_id = df_param_id.filter(col('nm_Item_Origem').isin(subtables_project)).collect()
                id_tabela = 'project_id'

            for data in data_list:
                list_ids.append(data['id'])

            for row in data_param_id:
                # Executa a função fn_StreamFromFolder_csv em uma thread do ThreadPoolExecutor
                subtask = executor.submit(get_tables_from_ids, *(row, baseUrl, dt_inicio_unix, dt_fim, location_landing, list_ids,id_tabela))
                # Adiciona a tarefa à lista de tarefas
                subtasks.append(subtask)

    except Exception as error:
        ## LOG ERRO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '

        if hasattr(error, 'code'): # captura o código do erro, caso possua o atributo 'code'
            error_code = error.code
        else:
            error_code = 'NULL'
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "cd_erro": error_code, "erro": str(error), "execUrl": execUrl}

        fn_LogFailed(parametros, dt_ingestao.strftime(format_log))

In [ ]:
# Cria uma lista para armazenar todas as tarefas
tasks = []
subtasks = []

# Cria uma instância do ThreadPoolExecutor com threads definidas (max_workers)
executor = concurrent.futures.ThreadPoolExecutor(max_workers=threads)

# Percorre todas as pastas do diretório de origem
for row in data_param:
    # Executa a função fn_StreamFromFolder_csv em uma thread do ThreadPoolExecutor
    task = executor.submit(get_tables, *(row, baseUrl, dt_inicio_unix, dt_fim, location_landing))
    # Adiciona a tarefa à lista de tarefas
    tasks.append(task)

# Aguarda a conclusão de todas as tarefas
_ = concurrent.futures.wait(tasks, return_when='ALL_COMPLETED')

executor.shutdown()

In [ ]:
tarefas = tasks + subtasks

for task in tarefas:
    try:
        print(task.result(),'\n')
    except Exception as error:
        print(error)
        pass